<a href="https://colab.research.google.com/github/sheetalojha/Neural-Style-Transfer/blob/master/NST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision.models import vgg19

In [ ]:
net = vgg19(pretrained = True).features
print(net)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth



Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPo

In [ ]:
for param in net.parameters():
  param.requires_grad = False


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [ ]:
%matplotlib inline
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
import requests
from torchvision import transforms, models

In [ ]:
def load_image(img_path, max_size = 400, shape = None):
  if "http" in img_path:
    response = requests.get(img_path)
    img = Image.open(BytesIO(response.content)).convert('RGB')
  else:
    image = Image.open(img_path).convert('RGB')
  
  if max(image.size) > max_size:
    size = max_size
  else:
    size = max(image.size)

  if shape is not None:
    size = shape
  in_transform = transform.Compose([transforms.Resize(size),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.486, 0.406),
                                                         (0.229, 0.224, 0.225))])
  image = in_transform(image)[:3, :, :].unsqueeze(0)

  return image

In [ ]:
#load in content and style image
content = load_image('images/octopus.jpg').to(device)
style = load_image('image/hockney.jpg', shape = content.shape[-2:]).to(device)

In [ ]:
#helper function for un-normalizing an image
#and converting it from a tensor image to a NumPy image for display
def im_convert(tensor):
  image = tensor.to("cpu").cpu().detach()
  image = image.numpy().squeeze()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.229, 0.224, 0.225)) + np.aaray((0.485, 0.456, 0.406))

  return image

In [ ]:
#display the images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
ax1.imshow(im_convert(content))
ax2.imshow(im_convert(style))


In [ ]:
def get_features(image, model, layers = None):
  if layers is None:
    layers = {'0': conv1_1,
              '1': conv2_1,
              '2': conv2_2,
              '3': conv3_1,
              '4': conv4_1,
              '4': conv4_2,
              '5': conv5_1,
              }
    features = {}
    x = image
    for name, layer in model.__model.items():
      x = layer(x)
      if name in layers:
        features[layers[name]] = x

    return features

In [ ]:
def gram_matrix(tensor):
  b, d, h, w = tensor.size()
  tensor.view(b, h*w)
  gram = torch.mm(tensor, tensor.t())

  return gram


In [ ]:
content_features = get_features(content, net)
style_features = get_features(style, net)

style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

target = content.clone().requires_grad_(True).to(device)

In [ ]:
style_weights = {'conv1_1': 1.,
                 'conv2_1': 0.8,
                 'conv3_1': 0.5,
                 'conv4_1': 0.3,
                 'conv5_1': 0.1}
content_weight = 1 #alpha
style_weight = 1e6 #beta

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
class conv_deconv(nn.Module):
    def __init__(self):
        super(conv_deconv, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 3, out_channels = 16, kernel_size = 4, stride = 1, padding = 0)
        nn.init.xavier_uniform(self.conv1.weight)
        self.conv2 = nn.Conv3d(in_channels = 16, out_channels = 32, kernel_size = 5)
        nn.init.xavier_uniform(self.conv2.weight)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool3d(kernel_size = (2,2,2), return_indices = True)
        
        self.deconv1 = nn.ConvTranspose3d(in_channels = 32, out_channels = 16, kernel_size = 5)
        nn.init.xavier_uniform(self.deconv1.weight)
        self.deconv2 = nn.ConvTranspose3d(in_channels = 16, out_channels = 3, kernel_size = 4)
        nn.init.xavier_uniform(self.deconv2.weight)
        self.maxunpool = nn.MaxUnpool3d(kernel_size = (2,2,2))
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        size1 = out.size()
        out, indices1 = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        #size2 = out.size()
        #out, indices2 = self.maxpool(out)
        
        out = self.deconv1(out)
        out = self.relu(out)
        out = self.maxunpool(out, indices1, size1)
        out = self.deconv2(out)
        out = self.relu(out)
        #out = self.maxunpool(out, indices1, size1)
        return(out)
        
      

In [ ]:
x = Variable(torch.rand(1, 3, 16, 224, 224))
net = conv_deconv()
output = net(x)
print(output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


tensor([[[[[0.0313, 0.0292, 0.0304,  ..., 0.0264, 0.0319, 0.0292],
           [0.0274, 0.0282, 0.0261,  ..., 0.0266, 0.0309, 0.0292],
           [0.0324, 0.0293, 0.0270,  ..., 0.0264, 0.0275, 0.0292],
           ...,
           [0.0255, 0.0285, 0.0188,  ..., 0.0283, 0.0291, 0.0292],
           [0.0292, 0.0292, 0.0305,  ..., 0.0291, 0.0291, 0.0292],
           [0.0292, 0.0292, 0.0292,  ..., 0.0292, 0.0292, 0.0292]],

          [[0.0278, 0.0279, 0.0261,  ..., 0.0262, 0.0314, 0.0292],
           [0.0274, 0.0262, 0.0289,  ..., 0.0279, 0.0293, 0.0292],
           [0.0275, 0.0242, 0.0302,  ..., 0.0291, 0.0273, 0.0292],
           ...,
           [0.0288, 0.0284, 0.0287,  ..., 0.0276, 0.0298, 0.0292],
           [0.0292, 0.0297, 0.0298,  ..., 0.0291, 0.0291, 0.0292],
           [0.0292, 0.0292, 0.0292,  ..., 0.0292, 0.0292, 0.0292]],

          [[0.0304, 0.0290, 0.0265,  ..., 0.0362, 0.0280, 0.0292],
           [0.0259, 0.0295, 0.0249,  ..., 0.0281, 0.0287, 0.0292],
           [0.0357, 0.0283

In [ ]:
def visualize_att(image_path, seq, alphas, rev_word_map, smooth=True):
    """
    Visualizes caption with weights at every word.
    Adapted from paper authors' repo: https://github.com/kelvinxu/arctic-captions/blob/master/alpha_visualization.ipynb
    :param image_path: path to image that has been captioned
    :param seq: caption
    :param alphas: weights
    :param rev_word_map: reverse word mapping, i.e. ix2word
    :param smooth: smooth weights?
    """
    image = Image.open(image_path)
    image = image.resize([14 * 24, 14 * 24], Image.LANCZOS)

    #words = [rev_word_map[ind] for ind in seq]

    for t in range(len(words)):
        if t > 50:
            break
        #plt.subplot(np.ceil(len(words) / 5.), 5, t + 1)

        #plt.text(0, 1, '%s' % (words[t]), color='black', backgroundcolor='white', fontsize=12)
        plt.imshow(image)
        #current_alpha = alphas[t, :]
        if smooth:
            alpha = skimage.transform.pyramid_expand(current_alpha.numpy(), upscale=24, sigma=8)
        else:
            alpha = skimage.transform.resize(current_alpha.numpy(), [14 * 24, 14 * 24])
        if t == 0:
            plt.imshow(alpha, alpha=0)
        else:
            plt.imshow(alpha, alpha=0.8)
        plt.set_cmap(cm.Greys_r)
        plt.axis('off')
    plt.show()

In [ ]:
import sklearn.manifold.TSNE
import torch
import torch.nn as nn
from torch.autograd import Variable
x = Variable(torch.rand(1, 1024))
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()